# Import Dependencies

In [ ]:
import pandas as pd

# Functions

In [ ]:
def print_unique_values(df, columns):
    for col in columns:
        print(f"\nColumn: {col}")
        unique_vals = df[col].unique()
        print(f"Number of unique values: {len(unique_vals)}")
        print("Unique values:")
        print(unique_vals)

def verify_cause_totals(df, year):
    # Filter to the specific year, Sex='All', and Age Group='All Ages'
    df_year = df[
        (df["Year"] == year) &
        (df["Sex"] == "All") &
        (df["Age Group"] == "[All]")
    ]

    all_causes_total = df_year[df_year["Indicator Name"] == "All Causes"]["Number"].sum()
    specific_causes_total = df_year[df_year["Indicator Name"] != "All Causes"]["Number"].sum()

    print(f"Year: {year}")
    print(f"Reported 'All Causes' total: {all_causes_total:,.0f}")
    print(f"Sum of specific causes:     {specific_causes_total:,.0f}")

    # Check if they match
    if abs(all_causes_total - specific_causes_total) < 1e-6:
        print("Match: Filtering is accurate.")
    else:
        print("Mismatch: Totals do not match.")

# Import Data

In [ ]:
# Load raw data
path = r"..\dataset\WHO Mortality Database - Overview of the distribution of causes of total deaths grouped by category - Malaysia - 30th May 2025 16_35.csv"
df = pd.read_csv(path, skiprows=8, usecols=range(10)) # Skip header metadata, keep first 10 columns only
# Check data columns
df.columns

In [ ]:
# Rename Columns to shorten
df.rename(columns={
    "Percentage of cause-specific deaths out of total deaths": "Percent of All Causes",
    "Age-standardized death rate per 100 000 standard population": "Age-standardized Death Rate",
    "Death rate per 100 000 population": "Death Rate"
}, inplace=True)

# Check renamed columns
df.columns

# Filter Redundant Columns

In [ ]:
# Remove redundant columns
columns=['Indicator Code', 'Age group code', 'Age-standardized Death Rate']
df.drop(columns, inplace=True, axis=1)
# After removed
df.head()

# Check for Missing Data

In [ ]:
# Missing data for each column
df.isnull().sum()

In [ ]:
# Get rows with missing values
pd.set_option("display.max_colwidth", None) # Display full text for columns text
df[df.isnull().any(axis=1)]

In [ ]:
# Remove unknown age group rowws with missing data
df = df[df["Age Group"] != "[Unknown]"]
# Remove footer metadata rows
df = df[~df["Indicator Name"].isin([
    "Usability",
    "Percentage of ill-defined or non-specific causes to total deaths",
    "Completeness"
])]

In [ ]:
# Check for missing values 
df.isnull().values.any()

# Check for Duplicate Data

In [ ]:
df.duplicated().sum()

# Check Data Consistency

In [ ]:
columns = ["Indicator Name", "Year", "Sex", "Age Group"]

print_unique_values(df, columns)

# Hierarchical Data Pruning

In [ ]:
# Define causes of death groups to remove
exclude_category = [
    'Ill-defined diseases',
    'Injuries',
    'Unintentional injuries',
    'Intentional injuries',
    'Ill-defined injuries/accidents',
    'Communicable, maternal, perinatal and nutritional conditions',
    'Maternal conditions',
    'Nutritional deficiencies',
    'Respiratory infections',
    'Infectious and parasitic diseases',
    'Perinatal conditions',
    'Noncommunicable diseases',
    'Cardiovascular diseases',
    'Malignant neoplasms',
    'Respiratory diseases',
    'Musculoskeletal diseases',
    'Genitourinary diseases',
    'Other neoplasms',
    'Sudden infant death syndrome',
    'Digestive diseases',
    'Neuropsychiatric conditions',
    'Diabetes mellitus and endocrine disorders',
    'Sense organ diseases',
    'Congenital anomalies',
    'Oral conditions',
    'Skin diseases'
]

# Filter the DataFrame
df = df[~df['Indicator Name'].isin(exclude_category)]
# View unique values
print_unique_values(df, ['Indicator Name'])

In [ ]:
verify_cause_totals(df, 2020)

# Feature Engineering Age Category & Population

In [ ]:
"""
Age group to category mapping:
- Infant: [0]
- Toddler: [1-4]
- Child: [5-14]
- Teenager: [15-19]
- Young Adult: [20-29]
- Adult: [30-44]
- Middle Age: [45-59]
- Senior: [60-79]
- Elderly: [80+]
- All Ages: [All]
"""

age_group_map = {
    '[0]': 'Infant',
    '[1-4]': 'Toddler',
    '[5-9]': 'Child',
    '[10-14]': 'Child',
    '[15-19]': 'Teenager',
    '[20-24]': 'Young Adult',
    '[25-29]': 'Young Adult',
    '[30-34]': 'Adult',
    '[35-39]': 'Adult',
    '[40-44]': 'Adult',
    '[45-49]': 'Middle Age',
    '[50-54]': 'Middle Age',
    '[55-59]': 'Middle Age',
    '[60-64]': 'Senior',
    '[65-69]': 'Senior',
    '[70-74]': 'Senior',
    '[75-79]': 'Senior',
    '[80-84]': 'Elderly',
    '[85+]': 'Elderly',
    '[All]': 'All Ages'
}

In [ ]:
# Create Age cateogroy column
df["Age Category"] = df["Age Group"].map(age_group_map)

# Reorder the columns
cols = df.columns.tolist()
cols.remove("Age Category")
age_group_index = cols.index("Age Group")
cols.insert(age_group_index + 1, "Age Category")
df = df[cols]
df

# Export Processed Data

In [ ]:
# Save the dataframe to CSV file
save_path = r"..\dataset\clean_data.csv"
df.to_csv(save_path, index=False)